# Machine Learning Model - Presentation 2
Group Members: Sanjay Krishna Moorthy , Viswashanthi Bonala , Mitchell Moreno , Sneha Dudyala , Susmitha Haripriya Varanasi

In [ ]:
# Spark init
!wget -q https://dlcdn.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz 
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/default-java"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"
!pip install -q findspark
import findspark
findspark.init()

In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
#import Packages
from pyspark.sql.types import *
from pyspark.sql.functions import split, col
import pyspark.sql.functions as func
from pyspark.sql.functions import length

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
custom_schema = StructType([
        StructField("ReviewId", IntegerType()),
        StructField("RecipeId", IntegerType()),
        StructField("AuthorId", IntegerType()),
        StructField("AuthorName", StringType()),
        StructField("Rating", IntegerType()),
        StructField("Review", StringType())
    ])

df_reviews = spark.read.csv('/content/drive/MyDrive/data/reviews.csv',schema = custom_schema, header=True)
df_reviews = df_reviews.drop("DateSubmitted")
df_reviews = df_reviews.drop("DateModified")

In [ ]:
df_reviews=df_reviews.na.drop()
df_reviews.show(5)

+--------+--------+--------+----------------+------+--------------------+
|ReviewId|RecipeId|AuthorId|      AuthorName|Rating|              Review|
+--------+--------+--------+----------------+------+--------------------+
|       2|     992|    2008|       gayg msft|     5|better than any y...|
|       7|    4384|    1634|   Bill Hilbrich|     4|I cut back on the...|
|       9|    4523|    2046|Gay Gilmore ckpt|     2|i think i did som...|
|      13|    7435|    1773|   Malarkey Test|     5|easily the best i...|
|      14|      44|    2085|      Tony Small|     5|  An excellent dish.|
+--------+--------+--------+----------------+------+--------------------+
only showing top 5 rows



In [ ]:
custom_schema2 = StructType([
        StructField("RecipeId", IntegerType()),
        StructField("Name", StringType()),
        StructField("AuthorId", StringType()),
        StructField("AuthorName", StringType()),
        StructField("CookTime", StringType()),
        StructField("PrepTime", StringType()),
        StructField("TotalTime", StringType()),
        StructField("DatePublished", StringType()),
        StructField("Description", StringType()),
        StructField("Images", StringType()),
        StructField("RecipeCategory", StringType()),
        StructField("Keywords", StringType()),
        StructField("RecipeIngredientQuantities", StringType()),
        StructField("RecipeIngredientParts", StringType()),
        StructField("AggregatedRating", FloatType()),
        StructField("ReviewCount", StringType()),
        StructField("Calories", FloatType()),
        StructField("FatContent", FloatType()),
        StructField("SaturatedFatContent", FloatType()),
        StructField("CholesterolContent", FloatType()),
        StructField("SodiumContent", FloatType()),
        StructField("CarbohydrateContent", FloatType()),
        StructField("FiberContent", FloatType()),
        StructField("SugarContent", FloatType()),
        StructField("ProteinContent", FloatType()),
        StructField("RecipeServings", StringType()),
        StructField("RecipeYield", StringType()),
        StructField("RecipeInstructions", StringType())
    ])
df_recipes = spark.read.csv('/content/drive/MyDrive/data/recipes.csv',schema = custom_schema2, header=True,escape='"')
df_recipes = df_recipes.drop("Images")
df_recipes = df_recipes.na.drop()

df_recipes.show(5)

+--------+--------------------+--------+--------------------+--------+--------+---------+--------------------+--------------------+--------------+--------------------+--------------------------+---------------------+----------------+-----------+--------+----------+-------------------+------------------+-------------+-------------------+------------+------------+--------------+--------------+------------+--------------------+
|RecipeId|                Name|AuthorId|          AuthorName|CookTime|PrepTime|TotalTime|       DatePublished|         Description|RecipeCategory|            Keywords|RecipeIngredientQuantities|RecipeIngredientParts|AggregatedRating|ReviewCount|Calories|FatContent|SaturatedFatContent|CholesterolContent|SodiumContent|CarbohydrateContent|FiberContent|SugarContent|ProteinContent|RecipeServings| RecipeYield|  RecipeInstructions|
+--------+--------------------+--------+--------------------+--------+--------+---------+--------------------+--------------------+-----------

In [ ]:
df_RecipeReviews = df_reviews.join(df_recipes, 'RecipeId') \
                             .select(df_reviews['ReviewId'],
                                     df_reviews['AuthorId'].alias('ReviewerId'),
                                     df_reviews['RecipeId'],
                                     df_recipes['Name'].alias('RecipeName'),
                                     df_reviews['Rating'],
                                     df_reviews['Review'],
                                     df_recipes['CookTime'],
                                     df_recipes['PrepTime'],
                                     df_recipes['TotalTime'],
                                     df_recipes['RecipeCategory'],
                                     df_recipes['AggregatedRating'],
                                     df_recipes['ReviewCount'].alias('RecipeReviewCount'),
                                     df_recipes['RecipeIngredientParts'],
                                     df_recipes['Calories'],
                                     df_recipes['FatContent'],
                                     df_recipes['SaturatedFatContent'],
                                     df_recipes['CholesterolContent'],
                                     df_recipes['SodiumContent'],
                                     df_recipes['CarbohydrateContent'],
                                     df_recipes['FiberContent'],
                                     df_recipes['SugarContent'],
                                     df_recipes['ProteinContent'],
                                     df_recipes['RecipeServings'],
                                    )

df_RecipeReviews.show(10)

+--------+----------+--------+--------------------+------+--------------------+--------+--------+---------+--------------+----------------+-----------------+---------------------+--------+----------+-------------------+------------------+-------------+-------------------+------------+------------+--------------+--------------+
|ReviewId|ReviewerId|RecipeId|          RecipeName|Rating|              Review|CookTime|PrepTime|TotalTime|RecipeCategory|AggregatedRating|RecipeReviewCount|RecipeIngredientParts|Calories|FatContent|SaturatedFatContent|CholesterolContent|SodiumContent|CarbohydrateContent|FiberContent|SugarContent|ProteinContent|RecipeServings|
+--------+----------+--------+--------------------+------+--------------------+--------+--------+---------+--------------+----------------+-----------------+---------------------+--------+----------+-------------------+------------------+-------------+-------------------+------------+------------+--------------+--------------+
|  137709|   

In [ ]:
# Changes the ingredients to an array and shows the schema and the dataframe
df_RecipeReviews = df_RecipeReviews.withColumn("RecipeIngredientParts", func.expr("substring(RecipeIngredientParts, 2)"))
df_RecipeReviews = df_RecipeReviews.withColumn("RecipeIngredientParts", func.translate(func.col("RecipeIngredientParts"), '()"', ''))
df_RecipeReviews = df_RecipeReviews.withColumn("RecipeIngredientParts", split(col('RecipeIngredientParts'), ','))
df_RecipeReviews.printSchema()
df_RecipeReviews.show(5, truncate = False)

root
 |-- ReviewId: integer (nullable = true)
 |-- ReviewerId: integer (nullable = true)
 |-- RecipeId: integer (nullable = true)
 |-- RecipeName: string (nullable = true)
 |-- Rating: integer (nullable = true)
 |-- Review: string (nullable = true)
 |-- CookTime: string (nullable = true)
 |-- PrepTime: string (nullable = true)
 |-- TotalTime: string (nullable = true)
 |-- RecipeCategory: string (nullable = true)
 |-- AggregatedRating: float (nullable = true)
 |-- RecipeReviewCount: string (nullable = true)
 |-- RecipeIngredientParts: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Calories: float (nullable = true)
 |-- FatContent: float (nullable = true)
 |-- SaturatedFatContent: float (nullable = true)
 |-- CholesterolContent: float (nullable = true)
 |-- SodiumContent: float (nullable = true)
 |-- CarbohydrateContent: float (nullable = true)
 |-- FiberContent: float (nullable = true)
 |-- SugarContent: float (nullable = true)
 |-- ProteinContent: float (n

# Creating Recipe Recommendation Engine

Recipe Rating ALS Recommendation Model

In [ ]:
index_df = df_RecipeReviews.select(df_RecipeReviews['RecipeID'], df_RecipeReviews['RecipeName'])

In [ ]:
reviews_indexed = df_RecipeReviews.select(df_RecipeReviews['ReviewerID'], df_RecipeReviews['RecipeID'], df_RecipeReviews['Rating'])

In [ ]:
numReviewers = reviews_indexed.select('ReviewerID').distinct().count()
print('The total number of reviewers is ' + str(numReviewers) + '.')

The total number of reviewers is 157500.


In [ ]:
numRecipes = reviews_indexed.select('RecipeID').distinct().count()
print('The total number of recipes is ' + str(numRecipes) + '.')

The total number of recipes is 229665.


In [ ]:
(training, test) = reviews_indexed.randomSplit([0.7, 0.3], seed = 1234)
print (training.first())
print('The number of records in the training data is ' + str(training.count()) + '.')
print('The number of records in the test data is ' + str(test.count()) + '.')

Row(ReviewerID=1533, RecipeID=44022, Rating=5)
The number of records in the training data is 559077.
The number of records in the test data is 239193.


In [ ]:
als = ALS(rank = 5, maxIter=20, regParam=0.1, userCol="ReviewerID", itemCol="RecipeID", ratingCol="Rating", coldStartStrategy="drop")
model = als.fit(training)

In [ ]:
predictions = model.transform(test)

In [ ]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="Rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.7335551874710295


In [ ]:
users = reviews_indexed.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = model.recommendForUserSubset(users,3)

In [ ]:
userSubsetRecs.show(truncate=False)

+----------+--------------------------------------------------------------+
|ReviewerID|recommendations                                               |
+----------+--------------------------------------------------------------+
|31261     |[{167605, 7.857949}, {151487, 7.5558033}, {188267, 7.3638124}]|
|286566    |[{163504, 6.6765213}, {141078, 6.467594}, {238218, 6.2725005}]|
|6357      |[{309777, 6.929102}, {141078, 6.9274054}, {167605, 6.90611}]  |
+----------+--------------------------------------------------------------+



In [ ]:
from pyspark.sql.functions import *
rating_recommendations = userSubsetRecs.withColumn('recomm_pair', explode('recommendations')) \
                                  .select('ReviewerID', col('recomm_pair.RecipeID'))
rating_recommendations

DataFrame[ReviewerID: int, RecipeID: int]

In [ ]:
recommended_recipes = rating_recommendations.join(index_df, 'RecipeID').distinct()

In [ ]:
rating_recommendations.show(10)

+----------+--------+
|ReviewerID|RecipeID|
+----------+--------+
|     31261|  167605|
|     31261|  151487|
|     31261|  188267|
|    286566|  163504|
|    286566|  141078|
|    286566|  238218|
|      6357|  309777|
|      6357|  141078|
|      6357|  167605|
+----------+--------+



In [ ]:
recommended_recipes.show(10)

+--------+----------+--------------------+
|RecipeID|ReviewerID|          RecipeName|
+--------+----------+--------------------+
|  309777|      6357|Dreamy, Creamy Pe...|
|  167605|     31261|Grilled Baby Vege...|
|  167605|      6357|Grilled Baby Vege...|
|  163504|    286566|Double Chocolate ...|
|  188267|     31261|Healthy Oatmeal B...|
|  238218|    286566| Easiest Iced Coffee|
|  141078|    286566|Country Fried Ste...|
|  141078|      6357|Country Fried Ste...|
|  151487|     31261|   Pain Au Chocolate|
+--------+----------+--------------------+



In [ ]:
df_RecipeReviews.filter((df_RecipeReviews['ReviewerID']==31261) & (df_RecipeReviews['Rating']==5) )[['ReviewerId','RecipeId','RecipeName','Rating']].orderBy(df_RecipeReviews['Rating'],asc = False).show(50)


+----------+--------+--------------------+------+
|ReviewerId|RecipeId|          RecipeName|Rating|
+----------+--------+--------------------+------+
|     31261|    8592|            BBQ Ribs|     5|
|     31261|   99899|Parmesan Oregano ...|     5|
|     31261|   16283|Mom's Tater Tots ...|     5|
|     31261|   20124|    Chicken in Gravy|     5|
|     31261|   23010|   Catalina Dressing|     5|
|     31261|   26309|           Meatballs|     5|
|     31261|   38769|       Cheese Straws|     5|
|     31261|   31710|Potato'n Salmon C...|     5|
|     31261|   14318|Spicy Salad Dressing|     5|
|     31261|   19821|      Linguini Salad|     5|
|     31261|   27344|    Homemade Noodles|     5|
|     31261|   16470|Homemade Drain Cl...|     5|
|     31261|   10776|      Scalloped Corn|     5|
|     31261|   65420|       Onion Burgers|     5|
|     31261|   77221|Southern Fluffy F...|     5|
|     31261|   88212|Spinach/Romaine S...|     5|
|     31261|   62053|       Crescent Dogs|     5|


# KNN model for Recommendations

In [ ]:
df_RecipeReviews.show(5, truncate = False)


+--------+----------+--------+---------------------+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+--------+---------+--------------+----------------+-----------------+-------------------------------------------------------+--------+----------+-------------------+------------------+-------------+-------------------+------------+------------+--------------+--------------+
|ReviewId|ReviewerId|RecipeId|RecipeName           |Rating|Review                                                                                                                         

In [ ]:
print(df_RecipeReviews.schema)

StructType(List(StructField(ReviewId,IntegerType,true),StructField(ReviewerId,IntegerType,true),StructField(RecipeId,IntegerType,true),StructField(RecipeName,StringType,true),StructField(Rating,IntegerType,true),StructField(Review,StringType,true),StructField(CookTime,StringType,true),StructField(PrepTime,StringType,true),StructField(TotalTime,StringType,true),StructField(RecipeCategory,StringType,true),StructField(AggregatedRating,FloatType,true),StructField(RecipeReviewCount,StringType,true),StructField(RecipeIngredientParts,ArrayType(StringType,true),true),StructField(Calories,FloatType,true),StructField(FatContent,FloatType,true),StructField(SaturatedFatContent,FloatType,true),StructField(CholesterolContent,FloatType,true),StructField(SodiumContent,FloatType,true),StructField(CarbohydrateContent,FloatType,true),StructField(FiberContent,FloatType,true),StructField(SugarContent,FloatType,true),StructField(ProteinContent,FloatType,true),StructField(RecipeServings,StringType,true)))


In [ ]:
df_RecipeReviews_features = df_RecipeReviews.select(df_RecipeReviews['ReviewId'], df_RecipeReviews['RecipeID'], df_RecipeReviews['Rating'],df_RecipeReviews['RecipeName'])
df_RecipeReviews_features.show(5)

+--------+--------+------+--------------------+
|ReviewId|RecipeID|Rating|          RecipeName|
+--------+--------+------+--------------------+
|  137709|     463|     0|Cappuccino Coffee...|
|  138400|     463|     5|Cappuccino Coffee...|
|  139158|     463|     5|Cappuccino Coffee...|
|  152220|     463|     5|Cappuccino Coffee...|
|  152268|     463|     5|Cappuccino Coffee...|
+--------+--------+------+--------------------+
only showing top 5 rows



In [ ]:
combine_recipe_rating = df_RecipeReviews_features.toPandas()


In [ ]:
recipe_ratingCount = (combine_recipe_rating.
     groupby(by = ['RecipeName'])['Rating'].
     count().
     reset_index().
     rename(columns = {'Rating': 'TotalRatingCount'})
     [['RecipeName', 'TotalRatingCount']]
    )
recipe_ratingCount.head()

,RecipeName,TotalRatingCount
0,"""A Bit Different"" Breakfast Pizza",4
1,"""AWW"" Marinated Olives",1
2,"""All in the Kitchen"" Chili",1
3,"""Alouette"" Potatoes",2
4,"""Amish"" tomato ketchup, for canning",1


In [ ]:
combine_recipe_rating.head()

,ReviewId,RecipeID,Rating,RecipeName
0,137709,463,0,Cappuccino Coffee Mix
1,138400,463,5,Cappuccino Coffee Mix
2,139158,463,5,Cappuccino Coffee Mix
3,152220,463,5,Cappuccino Coffee Mix
4,152268,463,5,Cappuccino Coffee Mix


In [ ]:
rating_with_totalRatingCount = combine_recipe_rating.merge(recipe_ratingCount, left_on = 'RecipeName', right_on = 'RecipeName', how = 'left')
rating_with_totalRatingCount.head()

,ReviewId,RecipeID,Rating,RecipeName,TotalRatingCount
0,137709,463,0,Cappuccino Coffee Mix,11
1,138400,463,5,Cappuccino Coffee Mix,11
2,139158,463,5,Cappuccino Coffee Mix,11
3,152220,463,5,Cappuccino Coffee Mix,11
4,152268,463,5,Cappuccino Coffee Mix,11


In [ ]:
#pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(recipe_ratingCount['TotalRatingCount'].describe())

count    202223.000000
mean          3.947474
std           6.706942
min           1.000000
25%           1.000000
50%           2.000000
75%           4.000000
max         404.000000
Name: TotalRatingCount, dtype: float64


In [ ]:
popularity_threshold = 50
rating_popular_recipe= rating_with_totalRatingCount.query('TotalRatingCount >= @popularity_threshold')
rating_popular_recipe.head()

,ReviewId,RecipeID,Rating,RecipeName,TotalRatingCount
68,71806,4519,4,Chicken Tortilla Soup,152
161,27935,9376,5,Beef Stroganoff,103
162,28729,9376,4,Beef Stroganoff,103
163,64494,9376,4,Beef Stroganoff,103
164,199334,9376,5,Beef Stroganoff,103


In [ ]:
rating_popular_recipe.shape

(50519, 5)

In [ ]:
## First lets create a Pivot matrix
recipe_features_df=rating_popular_recipe.pivot_table(index='RecipeName',columns='ReviewId',values='Rating').fillna(0)
recipe_features_df.head()

ReviewId,17,26,80,98,111,199,227,234,243,254,...,2089662,2089703,2089725,2089983,2090011,2090046,2090074,2090127,2090211,2090244
RecipeName,,,,,,,,,,,,,,,,,,,,,
"""Land of Nod"" Cinnamon Buns",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
&quot; Bar&quot; Cheese,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Get Up & Go' Bars,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15 Bean Soup,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
2- Handed Kitchen Sink Tomato Sandwich,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from scipy.sparse import csr_matrix

recipe_features_df_matrix = csr_matrix(recipe_features_df.values)

from sklearn.neighbors import NearestNeighbors


model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(recipe_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
recipe_features_df.shape

(677, 50519)

In [ ]:
import numpy as np
query_index = np.random.choice(recipe_features_df.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(recipe_features_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

446


In [ ]:
recipe_features_df.head()

ReviewId,17,26,80,98,111,199,227,234,243,254,...,2089662,2089703,2089725,2089983,2090011,2090046,2090074,2090127,2090211,2090244
RecipeName,,,,,,,,,,,,,,,,,,,,,
"""Land of Nod"" Cinnamon Buns",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
&quot; Bar&quot; Cheese,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Get Up & Go' Bars,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15 Bean Soup,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
2- Handed Kitchen Sink Tomato Sandwich,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(recipe_features_df.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, recipe_features_df.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for Oatmeal Carmelitas:

1: Oatmeal Pancakes, with distance of 1.0:
2: Olive Garden Salad Dressing, with distance of 1.0:
3: Oatmeal Waffles or Pancakes, with distance of 1.0:
4: Orange Chicken, with distance of 1.0:
5: Oatmeal Raisin Cookies, with distance of 1.0:
